# Data overview and parameter estimation
This notebook is used to get an overview of the video frames and iteratively estimate the necessary parameters for the gaussian blur and hough transformation. 

In [1]:
import cv2
import json
import os
from matplotlib import pyplot as plt
import numpy as np
import random
import sys

# add parent directory to python search path
sys.path.append(os.path.abspath('../'))

import config_helper

In [2]:
config = config_helper.read_config('../config.json')

In [3]:
files = []

for dirpath, dirnames, filenames in os.walk(config['data']):
    files.extend([os.path.join(dirpath, file) for file in filenames])

print('{} files found!'.format(len(files)))

850431 files found!


In [4]:
sample_files = random.sample(files, 9)

def show_images(images):
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    
    for i in range(3):
        for j in range(3):
            axes[i, j].imshow(images[3 * i + j])
            axes[i, j].axis('off')
            file_name = sample_files[3 * i + j].split('/')[-1]
            axes[i, j].text(0, -20, file_name, ha='left', size=10)
    
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()

In [ ]:
images = []

for file in sample_files:
    # Load image
    img = cv2.imread(file)
    
    # Convert to gray-scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Cut the image according to the config
    height, width = gray.shape
    gray = gray[
        config['preprocessing']['crop_top']:height - config['preprocessing']['crop_bottom'], 
        config['preprocessing']['crop_left']:width - config['preprocessing']['crop_right']
    ]
    
    # Add Gaussian blur
    gray = cv2.GaussianBlur(gray, (config['gaussian']['ksize'], config['gaussian']['ksize']), config['gaussian']['sigma'])
    
    images.append(gray)

show_images(images)

In [ ]:
hough_circles = []
masked_circles = []

half_width = int(config['sign_crop_width'] / 2)
half_height = int(config['sign_crop_height'] / 2)

for image in images:
    # Detect circles in the image using the Hough transform
    circles = cv2.HoughCircles(image, 
        cv2.HOUGH_GRADIENT, 
        config['hough']['dp'], 
        config['hough']['min_distance'], 
        param1=config['hough']['param1'], 
        param2=config['hough']['param2'], 
        minRadius=config['hough']['min_radius'], 
        maxRadius=config['hough']['max_radius'])

    # create rgb copy of the image
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    # If circles were detected
    if circles is not None:
        # Convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype('int')

        # Loop over the circles
        for (x, y, r) in circles:
            # Crop an image around the circle
            crop = image[y - half_height:y + half_height, x - half_width:x + half_width]

            # mask the background
            mask = np.zeros(crop.shape[:2], np.uint8)
            cv2.circle(mask, (half_width, half_height), r, (255, 255, 255), -1)
            masked = cv2.bitwise_and(crop, crop, mask=mask)

            masked_circles.append(masked)
        
        # Draw the circles in a seperate step, to avoid parts of the circles in the cropped images
        for (x, y, r) in circles:
            # Draw the circle on the image
            cv2.circle(image, (x, y), r, (0, 255, 0), 2)
    
    hough_circles.append(image)

show_images(hough_circles)

for image in masked_circles:
    plt.imshow(image)
    plt.show()

In [7]:
# save parameters to config file

with open('../config.json', 'w') as outfile:
    json.dump(config, outfile)